In [1]:
import torch
import os
import numpy as np
from dotenv import load_dotenv
from pinecone import Pinecone, PodSpec
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
import time
from huggingface_hub import InferenceClient

In [2]:
cache_dir="cache_dir"
load_dotenv('./.env')

True

In [3]:
device = torch.device('cuda') if torch.cuda.is_available() else 'cpu'
num_gpus = torch.cuda.device_count()

if num_gpus > 0:
    print(f"GPUs available: {num_gpus}")
    for i in range(num_gpus):
        gpu_name = torch.cuda.get_device_name(i)
        print(f"GPU {i}: {gpu_name}")
else:
    print("There is no GPU available. Using CPU")

GPUs available: 1
GPU 0: NVIDIA RTX A4000


#### Leitura do arquivo e criação do splitter'

In [4]:
with open('docs/nat.txt') as f:
    clt = f.read()

# Criação de um objeto RecursiveCharacterTextSplitter para dividir o texto em pedaços
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=10,
    length_function=len
)

In [5]:
chunks = text_splitter.create_documents([clt])
text_chunks = [doc.page_content for doc in chunks]

In [6]:
# Inicialize o modelo de embedding
model = SentenceTransformer('all-MiniLM-L6-v2')

# Isso retornará uma lista de vetores
embeddings = model.encode(text_chunks, convert_to_tensor=False)  

# Adiciona zeros para estender cada vetor até a dimensão 1536
extended_embeddings = np.array([np.pad(emb, (0, 1536 - len(emb)), 'constant') for emb in embeddings])
print(extended_embeddings[0].shape)

(1536,)


In [7]:
pc = Pinecone(api_key=os.environ.get('PINECONE_API_KEY'))
indexes = pc.list_indexes()
index_name = 'natgpt'

In [8]:
for i in indexes:
    pc.delete_index(i['name'])
    print('Index encontrada e apagada: ' + i['name'])

if index_name not in pc.list_indexes():
    pc.create_index(index_name, dimension=1536, metric='cosine', spec=PodSpec(environment=os.environ.get('PINECONE_ENV')))
    print('Index '+index_name+' criado')

Index encontrada e apagada: natgpt
Index natgpt criado


In [9]:
# Conecta na index criada
pc_index = pc.Index(index_name)

In [10]:
documents_to_insert = []
for i, embedding in enumerate(extended_embeddings):
    doc_id = f"{i}"
    documents_to_insert.append({"id": doc_id, "values": embedding.tolist()})

In [11]:
# Insere os documentos na index do Pinecone
pc_index.upsert(vectors=documents_to_insert)

# Aguarda 20 segundos para dar tempo de atualizar a Index no Pinecone
time.sleep(20)

### Busca por similaridade

In [12]:
# SentenceTransformer para gerar o embedding
model = SentenceTransformer('all-MiniLM-L6-v2')

# Inicialização do modelo para perguntas e respostas
llm = InferenceClient(model="mistralai/Mistral-7B-Instruct-v0.1")

In [13]:
def perguntar(prompt, top_k=3, debug=False):
    """
    Recebe um prompt do usuário para fazer uma busca semântica, encontra os resultados
    correspondentes no documento original e passa o resultado como contexto ao LLM

    Parâmetros:
    - prompt (str): Prompt do usuário. Também é usado para a busca semântica
    - top_k (int): Quantidade de resultados mais semalhantes
    - debug (bool): Retorna (ou não) o contexto passado ao LLM

    Retorna:
    - Tipo do retorno: A resposta do LLM com base na pergunta e contexto
    """
    
    # Gerando o vetor de consulta
    query_vector = model.encode(prompt, convert_to_tensor=False)
    
    # Adiciona zeros para estender cada vetor até a dimensão 1536
    padding_length = 1536 - len(query_vector)
    padded_vector = np.pad(query_vector, (0, padding_length), 'constant')
    
    query_vector_list = padded_vector.tolist()
    query_result = pc_index.query(vector=query_vector_list, top_k=top_k)
    
    contexto = []
    for index, match in enumerate(query_result.matches):
        contexto.append(text_chunks[int(match.id)])

    input_text = f"[INST]Responda em Português: {prompt}.\n\nContexto: {contexto}[/INST]"

    response = llm.text_generation(
        input_text,
        temperature=0.2, 
        max_new_tokens=500,
        top_k=30,
        top_p=0.9,
        repetition_penalty=1.0
    )

    if(debug == True):
        response = response + "\n\n\nDebug\n---------------------------------\n" + str(contexto)

    return response

### Perguntas com base no contexto passado

In [14]:
print(
    perguntar(
        prompt='Quem são os coordenadores do nat?', top_k=3, debug=False
    )
)

 Os coordenadores do NAT são Marcela Cristina Ozório como Coordenadora-Geral e Bernardo Fiterman Albano, como Coordenador-Adjunto.


In [15]:
print(
    perguntar(
        prompt='Qual o site do nat?', top_k=3, debug=False
    )
)

 O site do NAT é https://nat.mpac.mp.br.


In [16]:
print(
    perguntar(
        prompt='Quais os principais setores do NAT', top_k=3, debug=False
    )
)

 Os principais setores do NAT são:

1. Coordenação de Desenvolvimento de Sistemas
2. Observatório Criminal
3. LAB
4. Observatório de Políticas Públicas
5. Técinco-Científica

Os coordenadores do NAT são Marcela Cristina Ozório como Coordenadora-Geral e Bernardo Fiterman Albano, como Coordenador-Adjunto. Mais informações sobre o NAT podem ser encontradas no site https://nat.mpac.mp.br. Por meio do SIGEP, os membros solicitantes poderão acompanhar a tramitação do seu pedido de apoio no NAT, uma vez que ficarão registradas todas as etapas dos trabalhos executados e documentos anexados, de modo a facilitar a localização dos processos e posicionamento tanto do demandante quanto da Coordenação. O acesso para registro e acompanhamento de pedidos de apoio ao NAT pode ser feito por meio do endereço: https://sigep.mpac.mp.br.


In [17]:
print(
    perguntar(
        prompt='Me da um resumo do NAT?', top_k=3, debug=False
    )
)

 O NAT (Núcleo de Apoio ao Processo de Pesquisa) é um órgão da Polícia Militar do Paraná (MPAC) responsável pela coordenação e suporte ao trabalho de pesquisa policial. O NAT tem vários setores, incluindo a Coordenação de Desenvolvimento de Sistemas, o Observatório Criminal, o LAB (Laboratório de Análises Biológicas), o Observatório de Políticas Públicas e a Técnica-Científica.

Os coordenadores do NAT são Marcela Cristina Ozório e Bernardo Fiterman Albano. O acesso para registro e acompanhamento de pedidos de apoio ao NAT pode ser feito por meio do endereço: https://sigep.mpac.mp.br. Por meio do SIGEP (Sistema de Gestão Eletrônica de Processos), os membros solicitantes poderão acompanhar a tramitação do seu pedido de apoio no NAT, uma vez que ficarão registradas todas as etapas dos trabalhos executados e documentos anexados, de modo a facilitar a localização dos processos e o posicionamento tanto do demandante quanto da Coordenação. Mais informações sobre o NAT podem ser encontradas n

In [18]:
print(
    perguntar(
        prompt='Me da um resumo do Observatório de Políticas Público', top_k=3, debug=False
    )
)

 O Observatório de Políticas Públicas (OPP) é um projeto do Núcleo de Apoio Técnico-NAT que se propõe a monitorar os orçamentos públicos e a aplicação dos recursos, com o objetivo de verificar a consonância entre o que foi planejado com o executado, se o resultado almejado foi alcançado e promoveu a transformação social que era buscada. O OPP usará instrumentos formais como o Plano Plurianual, Lei de Diretrizes Orçamentárias e Lei Orçamentária Anual. O OPP produzirá e disponibilizará dados, informações e conhecimentos aos membros do MPAC, estabelecerá um sistema de informação confiável, flexível e escalável, e abrangerá as áreas da saúde, educação, meio ambiente, segurança pública e políticas de assistência social.


In [19]:
print(
    perguntar(
        prompt='Qual o site do OPP?', top_k=3, debug=False
    )
)

 O site do OPP (Observatório Político Público) é https://nat.mpac.mp.br/posts/category/observatorio-politicas-publicas/.


In [20]:
# Pergunta com resposta totalmente fora do contexto
print(
    perguntar(
        prompt='Me da um resumo, usando markdown, do que são redes neurais artificiais', top_k=3, debug=False
    )
)

 Redes neurais artificiais são sistemas computacionais que tentam simular o funcionamento do cérebro humano. São utilizados em diversos campos, como a inteligência artificial, a saúde e a educação. Esses sistemas são capazes de aprender e adaptar-se ao seu ambiente, e podem ser utilizados para resolver problemas complexos, como a identificação de padrões em dados ou a tradução de idiomas.
